The following is a data science illustration of The foundational paper on FDFPGs. This is meant to be part one of a two part paper review, with a future modeling of Quadratic Funding and Matching Funds Requirments by Pasquini at Gitcoin. 

This worik is for educational purposes in the domain of token engineering. This work is meant to illustrate a data science approach to systems based modeling. This work is in the domain of digital signal processing, with primary foundations in machine learning, big data, data science, economics, and computing science, as those are the backgrounds of the primary authors. This work is funded by  with the Token Engineering Commons and Griff Green. 

# A Flexible Design for Funding Public Goods

Bhuterin, Hitzig, Weyl on A Flexible Design for Funding Public Goods. 

https://arxiv.org/pdf/1809.06421.pdf

The following is a data science illustration of The foundational paper on FDFPGs. This is meant to be part one of a two part paper review, with a future modeling of Quadratic Funding and Matching Funds Requirments by Pasquini at Gitcoin. 

This worik is for educational purposes in the domain of token engineering. This work is meant to illustrate a data science approach to systems based modeling. This work is in the domain of digital signal processing, with primary foundations in machine learning, big data, data science, economics, and computing science, as those are the backgrounds of the primary authors. This work is funded by  with the Token Engineering Commons and Griff Green. 

## 3.0 Model

* The Foundational Model
* Society, Citizens, Community, and Public Goods.

In [ ]:
import numpy as np

# Number of Citizens in the Society
N = 30

# Max Community Size
C = 20

# Max Number of Public Goods
P = 10

# Society is a set of citizens
society = list(range(N))

# Community is a random subset of the society. The community size is from 25 up to size of the society.
community = np.random.choice(a=list(society), size=C, replace=False, p=None)

# Public Goods are proposed by community members. Cardinality is from 20 up to size of the community.
public_goods = list(enumerate(np.random.choice(a=list(community), size=P, replace=True, p=None)))

Community members are sampled from society.

In [ ]:
community

array([16, 26, 17, 28, 13, 25, 24, 19, 12, 20, 21, 23,  6,  0, 29,  2, 11,
        1,  9,  4])

Public goods are sampled from the community with replace=True ie. a community member can steward multiple public goods.

In [ ]:
public_goods

[(0, 19),
 (1, 20),
 (2, 20),
 (3, 26),
 (4, 29),
 (5, 20),
 (6, 9),
 (7, 6),
 (8, 17),
 (9, 16)]

In [ ]:
len(society)

30

In [ ]:
len(community)

20

In [ ]:
len(public_goods)

10

## 3.1 Individual Preferences and Actions
* Value Function Generators
* Value Function Dataset
* Visualizing Value Functions
* Public Goods Generators
* Citizen Contribution Generators
* The Contributions Dataset
* The Utility of Citizens

### Value Function Generators

The paper calls for 
* Concave, 
* Smooth, 
* Increasing preferences towards the funding of public goods.

The following function generator, generates value functions in the domain of {[0,1],[0,1]}.

In [ ]:
import param
import numpy as np
import panel as pn
import hvplot.pandas
import pandas as pd

class ConcaveFunctionGenerator(param.Parameterized):
    f0 = param.Number(default=0.2, bounds=(0, 1), doc="Value of f(0)")
    f1 = param.Number(default=0.8, bounds=(0, 1), softbounds=(0, 1), doc="Value of f(1)")
    steepness = param.Number(default=5, bounds=(1, 20), doc="Steepness of the curve")

    def __init__(self, **params):
        super().__init__(**params)
        self._update_f1_bounds()

    @param.depends('f0', watch=True)
    def _update_f1_bounds(self):
        # Clip the value of f1 if it's below f0
        self.f1 = max(self.f0, self.f1)
        
        # Update the lower bound of f1 to be the value of f0
        self.param['f1'].bounds = (self.f0, 1)
        
    def x(self):
        return np.linspace(0, 1, 400)

    @param.depends('f0', 'f1', 'steepness')
    def f(self, x):
        # Using the negative exponential function as a base
        y = 1 - np.exp(-self.steepness * x)
        
        # Adjusting the function to start at f0 and end at f1
        y = self.f0 + (self.f1 - self.f0) * (y - y.min()) / (y.max() - y.min())
        
        return y

    @param.depends('f0', 'f1', 'steepness')
    def view(self):
        x = self.x()
        y = self.f(x)
        df = pd.DataFrame({'x': x, 'y': y})
        return df.hvplot.line(x='x', y='y', ylim=(0, 1.01))

concave_gen = ConcaveFunctionGenerator()
pn.Row(concave_gen.param, concave_gen.view).servable()


Row
    [0] Column(margin=(5, 10), name='ConcaveFunctionGenerator'...)
        [0] StaticText(value='<b>ConcaveFunctionGenerat...)
        [1] FloatSlider(name='F0', value=0.2)
        [2] FloatSlider(name='F1', start=0.2, value=0.8)
        [3] FloatSlider(end=20, name='Steepness', start=1, value=5)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

The concave value generator has the following properties:
1. f0 = f(0) is in [0,1]
2. f1 = f(1) is in [f0,1]
3. f(x) = 1 - e**(-s*x)
4. The negative exponential function provides a smooth, concave, increasing function from f0 to f1 with a parameterized steepness.

The class dynamically clips f1 to satisfy property 2. Such as in the following example:

In [ ]:
ConcaveFunctionGenerator(f0=1,f1=0)

ConcaveFunctionGenerator(f0=1, f1=1, name='ConcaveFunctionGenerator12080', steepness=5)

Here we propose parameter generators for sampling concave functions

In [ ]:
import numbergen as ng
import numpy as np


# For CurveGenerator
def concave_function_parameters_generator():
    return dict(
        f0=ng.BoundedNumber(generator=ng.NormalRandom(mu=0.1, sigma=0.3), bounds=(0,1))(),
        f1=ng.BoundedNumber(generator=ng.NormalRandom(mu=0.5, sigma=0.4), bounds=(0,1))(),
        steepness=ng.UniformRandom(lbound=1, ubound=20)(),
    )

This method generates random parameters given the distributions described for f0, f1, and steepness above. 
* f0 is a normal distribution at (0.1,0.3), clipped at (0,1)
* f0 is a normal distribution at (0.5,0.4), clipped at (0,1)
* steepness is a uniform distribution at (1,20)

In [ ]:
concave_function_parameters_generator()

{'f0': 0, 'f1': 1, 'steepness': 4.00239305661294}

In [ ]:
value_functions = [ConcaveFunctionGenerator(**concave_function_parameters_generator()) for p_i in range(len(public_goods)*len(society))]

Here we can see all of the preferences of the citizens. These are the value functions V_p_i(F_p). There is one preference function for each V_p_i in {public_goods}X{citizens}. Cardinality is len(public_goods)*len(citizens).

#### Visualizing Sampled Value Functions

In [ ]:
import pandas as pd

pd.DataFrame([s.param.values() for s in value_functions])

sample_p_i_slider = pn.widgets.IntSlider(name='Utility Value Function', start=0, end=len(value_functions)-1)

pn.Row(sample_p_i_slider, pn.bind(lambda i: value_functions[i].view(), i=sample_p_i_slider))

Row
    [0] IntSlider(end=299, name='Utility Value Function')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

#### Value Functions Dataset

Create a Dataframe! These are citizen preferences by funding amounts for public goods. Take Funding Amount as the Index by applying a transpose. 

In [ ]:
df_value_functions = pd.DataFrame([s.f(s.x()) for s in value_functions])
df_value_functions = df_value_functions.T
df_value_functions.shape

(400, 300)

Make Funding the Index from 0 to 100

In [ ]:
df_value_functions.index = np.linspace(0,1,len(df_value_functions))
df_value_functions.index.name = "funding"

Label the Columns by {public_goods}X{Citizens}.

In [ ]:
df_value_functions.columns = [(p, i) for p in public_goods for i in society]
df_value_functions.columns.name = "value_p_i"

Number of columns is len(public_goods)*len(society).

In [ ]:
df_value_functions

value_p_i,"((0, 19), 0)","((0, 19), 1)","((0, 19), 2)","((0, 19), 3)","((0, 19), 4)","((0, 19), 5)","((0, 19), 6)","((0, 19), 7)","((0, 19), 8)","((0, 19), 9)",...,"((9, 16), 20)","((9, 16), 21)","((9, 16), 22)","((9, 16), 23)","((9, 16), 24)","((9, 16), 25)","((9, 16), 26)","((9, 16), 27)","((9, 16), 28)","((9, 16), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,0.0,0.621021,0.000000,0.000000,0.000000,0.244184,0.2604,0.329710,0.000000,0.000000,...,0.013492,0.335301,0.311463,0.000000,0.000000,0.000000,0.000000,0.046995,0.038176,0.651195
0.002506,0.0,0.624531,0.026909,0.021756,0.012057,0.258017,0.2604,0.343405,0.003232,0.008038,...,0.037561,0.337664,0.323878,0.035220,0.012811,0.021808,0.006475,0.075161,0.050954,0.651195
0.005013,0.0,0.627956,0.052751,0.042838,0.023802,0.271528,0.2604,0.356679,0.006396,0.015842,...,0.060883,0.339946,0.335871,0.069199,0.025411,0.042961,0.012872,0.102146,0.063562,0.651195
0.007519,0.0,0.631300,0.077567,0.063266,0.035245,0.284726,0.2604,0.369544,0.009495,0.023420,...,0.083481,0.342150,0.347456,0.101981,0.037804,0.063477,0.019193,0.127999,0.076005,0.651195
0.010025,0.0,0.634563,0.101399,0.083060,0.046391,0.297617,0.2604,0.382014,0.012530,0.030777,...,0.105379,0.344279,0.358648,0.133609,0.049993,0.083377,0.025438,0.152767,0.088282,0.651195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.0,0.767149,0.678194,0.701840,0.467019,0.839827,0.2604,0.775061,0.155717,0.276450,...,0.789752,0.404261,0.676702,1.000000,0.778474,0.725126,0.536586,0.718313,0.999739,0.651195
0.992481,0.0,0.767149,0.678194,0.701840,0.467019,0.839828,0.2604,0.775061,0.155718,0.276450,...,0.789752,0.404261,0.676702,1.000000,0.778493,0.725127,0.536642,0.718313,0.999805,0.651195
0.994987,0.0,0.767149,0.678194,0.701840,0.467020,0.839830,0.2604,0.775061,0.155719,0.276450,...,0.789752,0.404261,0.676702,1.000000,0.778511,0.725127,0.536697,0.718313,0.999871,0.651195


In [ ]:
df_value_functions_melted = df_value_functions.melt(ignore_index=False)
df_value_functions_melted['public_good'] = df_value_functions_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[0]).astype(str)
df_value_functions_melted['citizen'] = df_value_functions_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[1]).astype(str)
df_value_functions_melted

,value_p_i,value,public_good,citizen
funding,,,,
0.000000,"((0, 19), 0)",0.000000,"(0, 19)",0
0.002506,"((0, 19), 0)",0.000000,"(0, 19)",0
0.005013,"((0, 19), 0)",0.000000,"(0, 19)",0
0.007519,"((0, 19), 0)",0.000000,"(0, 19)",0
0.010025,"((0, 19), 0)",0.000000,"(0, 19)",0
...,...,...,...,...
0.989975,"((9, 16), 29)",0.651195,"(9, 16)",29
0.992481,"((9, 16), 29)",0.651195,"(9, 16)",29
0.994987,"((9, 16), 29)",0.651195,"(9, 16)",29


#### V_p(F_p)

In [ ]:
vpfp = df_value_functions_melted.groupby(['funding', 'public_good'])[['value']].sum().reset_index()

In [ ]:
vpfp

,funding,public_good,value
0,0.0,"(0, 19)",4.048026
1,0.0,"(1, 20)",4.478996
2,0.0,"(2, 20)",5.402273
3,0.0,"(3, 26)",6.100383
4,0.0,"(4, 29)",5.265308
...,...,...,...
3995,1.0,"(5, 20)",16.155008
3996,1.0,"(6, 9)",18.412522
3997,1.0,"(7, 6)",16.289352
3998,1.0,"(8, 17)",13.368421


In [ ]:
vpfp = vpfp.pivot_table(columns='public_good', values='value', index='funding')
vpfp

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,4.048026,4.478996,5.402273,6.100383,5.265308,4.920779,5.885565,5.825851,6.019628,4.427496
0.002506,4.392000,4.715154,5.714727,6.433432,5.482943,5.167939,6.208467,6.069587,6.169991,4.749682
0.005013,4.725573,4.943942,6.015571,6.753896,5.694758,5.408208,6.521046,6.306050,6.316433,5.062714
0.007519,5.049086,5.165616,6.305260,7.062288,5.900935,5.641810,6.823673,6.535492,6.459073,5.366876
0.010025,5.362870,5.380428,6.584232,7.359099,6.101647,5.868960,7.116706,6.758155,6.598024,5.662442
...,...,...,...,...,...,...,...,...,...,...
0.989975,16.935890,14.077718,14.543706,16.481353,15.694245,16.148642,18.408543,16.285519,13.363597,17.440689
0.992481,16.936358,14.079070,14.543760,16.482066,15.695607,16.150246,18.409550,16.286490,13.364814,17.441273
0.994987,16.936821,14.080414,14.543812,16.482776,15.696959,16.151842,18.410548,16.287452,13.366023,17.441851


In [ ]:
marginal_value = vpfp.diff().div(vpfp.index.to_series().diff(), axis=0).bfill()
marginal_value

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,137.245932,94.227340,124.669084,132.886843,86.836316,98.616775,128.838100,97.250623,59.994966,128.552089
0.002506,137.245932,94.227340,124.669084,132.886843,86.836316,98.616775,128.838100,97.250623,59.994966,128.552089
0.005013,133.095582,91.286130,120.036764,127.865066,84.514379,95.867338,124.718923,94.348684,58.430540,124.899895
0.007519,129.081688,88.448198,115.586085,123.048293,82.264443,93.207154,120.748144,91.547324,56.913233,121.360670
0.010025,125.199490,85.709669,111.309672,118.427806,80.084084,90.633021,116.920046,88.842735,55.441502,117.930723
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.188006,0.543048,0.021596,0.286348,0.547275,0.643686,0.404705,0.390829,0.488078,0.235331
0.992481,0.186462,0.539536,0.021299,0.284786,0.543298,0.640209,0.401525,0.387418,0.485282,0.233037
0.994987,0.184934,0.536048,0.021007,0.283238,0.539356,0.636758,0.398374,0.384038,0.482506,0.230767


In [ ]:
positive_marginal_value = marginal_value.where(marginal_value > 1, 0)
positive_marginal_value

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,137.245932,94.227340,124.669084,132.886843,86.836316,98.616775,128.838100,97.250623,59.994966,128.552089
0.002506,137.245932,94.227340,124.669084,132.886843,86.836316,98.616775,128.838100,97.250623,59.994966,128.552089
0.005013,133.095582,91.286130,120.036764,127.865066,84.514379,95.867338,124.718923,94.348684,58.430540,124.899895
0.007519,129.081688,88.448198,115.586085,123.048293,82.264443,93.207154,120.748144,91.547324,56.913233,121.360670
0.010025,125.199490,85.709669,111.309672,118.427806,80.084084,90.633021,116.920046,88.842735,55.441502,117.930723
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
individual_marginal_value = df_value_functions.diff().div(df_value_functions.index.to_series().diff(), axis=0).bfill()
individual_marginal_value

value_p_i,"((0, 19), 0)","((0, 19), 1)","((0, 19), 2)","((0, 19), 3)","((0, 19), 4)","((0, 19), 5)","((0, 19), 6)","((0, 19), 7)","((0, 19), 8)","((0, 19), 9)",...,"((9, 16), 20)","((9, 16), 21)","((9, 16), 22)","((9, 16), 23)","((9, 16), 24)","((9, 16), 25)","((9, 16), 26)","((9, 16), 27)","((9, 16), 28)","((9, 16), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,0.0,1.400432,10.736806,8.680624,4.810643,5.519203,0.0,5.464316,1.289458,3.207199,...,9.603183,9.429517e-01,4.953626,14.052626,5.111454,8.701567,2.583460,1.123849e+01,5.098275,0.0
0.002506,0.0,1.400432,10.736806,8.680624,4.810643,5.519203,0.0,5.464316,1.289458,3.207199,...,9.603183,9.429517e-01,4.953626,14.052626,5.111454,8.701567,2.583460,1.123849e+01,5.098275,0.0
0.005013,0.0,1.366798,10.310793,8.411540,4.686454,5.391043,0.0,5.296283,1.262703,3.113947,...,9.305435,9.106365e-01,4.785244,13.557698,5.027460,8.439865,2.552555,1.076696e+01,5.030880,0.0
0.007519,0.0,1.333971,9.901682,8.150796,4.565470,5.265859,0.0,5.133417,1.236504,3.023407,...,9.016919,8.794287e-01,4.622586,13.080202,4.944847,8.186035,2.522019,1.031521e+01,4.964376,0.0
0.010025,0.0,1.301932,9.508805,7.898135,4.447610,5.143581,0.0,4.975560,1.210848,2.935499,...,8.737348,8.492904e-01,4.465456,12.619522,4.863591,7.939838,2.491849,9.882408e+00,4.898752,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.0,0.000097,0.000001,0.000036,0.000161,0.000527,0.0,0.000025,0.000333,0.000029,...,0.000039,1.017634e-06,0.000006,0.000010,0.007471,0.000052,0.022537,5.204252e-07,0.026940,0.0
0.992481,0.0,0.000095,0.000001,0.000034,0.000157,0.000515,0.0,0.000024,0.000326,0.000028,...,0.000038,9.827590e-07,0.000006,0.000010,0.007349,0.000050,0.022267,4.985896e-07,0.026583,0.0
0.994987,0.0,0.000092,0.000001,0.000033,0.000153,0.000503,0.0,0.000023,0.000320,0.000027,...,0.000037,9.490796e-07,0.000006,0.000010,0.007228,0.000049,0.022001,4.776702e-07,0.026232,0.0


In [ ]:
positive_individual_marginal_value = individual_marginal_value.where(individual_marginal_value > 1, 0)
positive_individual_marginal_value

value_p_i,"((0, 19), 0)","((0, 19), 1)","((0, 19), 2)","((0, 19), 3)","((0, 19), 4)","((0, 19), 5)","((0, 19), 6)","((0, 19), 7)","((0, 19), 8)","((0, 19), 9)",...,"((9, 16), 20)","((9, 16), 21)","((9, 16), 22)","((9, 16), 23)","((9, 16), 24)","((9, 16), 25)","((9, 16), 26)","((9, 16), 27)","((9, 16), 28)","((9, 16), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,0.0,1.400432,10.736806,8.680624,4.810643,5.519203,0.0,5.464316,1.289458,3.207199,...,9.603183,0.0,4.953626,14.052626,5.111454,8.701567,2.583460,11.238493,5.098275,0.0
0.002506,0.0,1.400432,10.736806,8.680624,4.810643,5.519203,0.0,5.464316,1.289458,3.207199,...,9.603183,0.0,4.953626,14.052626,5.111454,8.701567,2.583460,11.238493,5.098275,0.0
0.005013,0.0,1.366798,10.310793,8.411540,4.686454,5.391043,0.0,5.296283,1.262703,3.113947,...,9.305435,0.0,4.785244,13.557698,5.027460,8.439865,2.552555,10.766957,5.030880,0.0
0.007519,0.0,1.333971,9.901682,8.150796,4.565470,5.265859,0.0,5.133417,1.236504,3.023407,...,9.016919,0.0,4.622586,13.080202,4.944847,8.186035,2.522019,10.315206,4.964376,0.0
0.010025,0.0,1.301932,9.508805,7.898135,4.447610,5.143581,0.0,4.975560,1.210848,2.935499,...,8.737348,0.0,4.465456,12.619522,4.863591,7.939838,2.491849,9.882408,4.898752,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
0.992481,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
0.994987,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [ ]:
individual_marginal_value_melted = positive_individual_marginal_value.melt(ignore_index=False)
individual_marginal_value_melted['public_good'] = individual_marginal_value_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[0]).astype(str)
individual_marginal_value_melted['citizen'] = individual_marginal_value_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[1]).astype(str)
individual_marginal_value_melted

,value_p_i,value,public_good,citizen
funding,,,,
0.000000,"((0, 19), 0)",0.0,"(0, 19)",0
0.002506,"((0, 19), 0)",0.0,"(0, 19)",0
0.005013,"((0, 19), 0)",0.0,"(0, 19)",0
0.007519,"((0, 19), 0)",0.0,"(0, 19)",0
0.010025,"((0, 19), 0)",0.0,"(0, 19)",0
...,...,...,...,...
0.989975,"((9, 16), 29)",0.0,"(9, 16)",29
0.992481,"((9, 16), 29)",0.0,"(9, 16)",29
0.994987,"((9, 16), 29)",0.0,"(9, 16)",29


In [ ]:
individual_marginal_value = individual_marginal_value_melted.pivot_table(index='funding', columns='public_good', values='value', aggfunc='sum')
individual_marginal_value

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,136.391487,91.159268,124.183756,131.241093,84.822605,96.209931,127.813200,95.849297,56.372315,126.116123
0.002506,136.391487,91.159268,124.183756,131.241093,84.822605,96.209931,127.813200,95.849297,56.372315,126.116123
0.005013,132.246958,88.255695,119.557861,126.230760,81.573590,93.502552,123.705367,92.967161,54.833964,122.519726
0.007519,128.238834,85.454853,115.113523,121.425330,79.406640,90.883490,119.745796,90.185320,53.342443,119.034684
0.010025,123.362937,82.752877,110.843367,116.816085,77.306165,88.349568,115.928774,87.499966,51.896212,115.657355
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
take_positive_index = lambda marginal_value: marginal_value.apply(lambda col: col[col != 0].last_valid_index()).replace(np.nan, 0)

In [ ]:
optimal_funding = take_positive_index(positive_marginal_value)
optimal_funding

public_good
(0, 19)    0.586466
(1, 20)    0.759398
(2, 20)    0.411028
(3, 26)    0.561404
(4, 29)    0.791980
(5, 20)    0.804511
(6, 9)     0.721805
(7, 6)     0.729323
(8, 17)    0.711779
(9, 16)    0.646617
dtype: float64

In [ ]:
private_contributions_funding = take_positive_index(individual_marginal_value)
private_contributions_funding

public_good
(0, 19)    0.305764
(1, 20)    0.255639
(2, 20)    0.208020
(3, 26)    0.210526
(4, 29)    0.348371
(5, 20)    0.280702
(6, 9)     0.348371
(7, 6)     0.370927
(8, 17)    0.250627
(9, 16)    0.308271
dtype: float64

In [ ]:
positive_marginal_value.hvplot.area()

:NdOverlay   [public_good]
   :Area   [funding]   (value,Baseline)

In [ ]:
individual_marginal_value.hvplot.area()

:NdOverlay   [public_good]
   :Area   [funding]   (value,Baseline)

In [ ]:
positive_marginal_value.sum(axis=1).hvplot.line(label='Collective Marginal Value', title="Suboptimality of Private Contributions") * individual_marginal_value.sum(axis=1).hvplot.line(label='Private Marginal Value')

:Overlay
   .Curve.Collective_Marginal_Value :Curve   [funding]   (0)
   .Curve.Private_Marginal_Value    :Curve   [funding]   (0)

### Definition 4 (1P1V Mechanisms)
Majority voting determines whether to fund each public good and the goods selected receive funding through taxes and transfers.

In [ ]:
individual_marginal_value

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,136.391487,91.159268,124.183756,131.241093,84.822605,96.209931,127.813200,95.849297,56.372315,126.116123
0.002506,136.391487,91.159268,124.183756,131.241093,84.822605,96.209931,127.813200,95.849297,56.372315,126.116123
0.005013,132.246958,88.255695,119.557861,126.230760,81.573590,93.502552,123.705367,92.967161,54.833964,122.519726
0.007519,128.238834,85.454853,115.113523,121.425330,79.406640,90.883490,119.745796,90.185320,53.342443,119.034684
0.010025,123.362937,82.752877,110.843367,116.816085,77.306165,88.349568,115.928774,87.499966,51.896212,115.657355
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
vote_results = individual_marginal_value_melted.groupby(['funding', 'public_good'])['value'].median().to_frame().pivot_table(index='funding', columns='public_good', values='value')
vote_results

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,4.375570,1.864483,3.947530,2.348807,2.106296,2.603769,3.280980,2.611261,1.428337,2.853014
0.002506,4.375570,1.864483,3.947530,2.348807,2.106296,2.603769,3.280980,2.611261,1.428337,2.853014
0.005013,4.284058,1.839549,3.797053,2.312952,2.061017,2.551248,3.220352,2.560260,1.384456,2.790209
0.007519,4.194588,1.815046,3.652662,2.277648,2.017257,2.499788,3.161164,2.510575,1.342100,2.729175
0.010025,4.107112,1.790966,3.514100,2.242888,1.974960,2.449367,3.103377,2.462166,1.301210,2.669857
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Vote Results

In [ ]:
(vote_results > 1).astype(int)

public_good,"(0, 19)","(1, 20)","(2, 20)","(3, 26)","(4, 29)","(5, 20)","(6, 9)","(7, 6)","(8, 17)","(9, 16)"
funding,,,,,,,,,,
0.000000,1,1,1,1,1,1,1,1,1,1
0.002506,1,1,1,1,1,1,1,1,1,1
0.005013,1,1,1,1,1,1,1,1,1,1
0.007519,1,1,1,1,1,1,1,1,1,1
0.010025,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
0.989975,0,0,0,0,0,0,0,0,0,0
0.992481,0,0,0,0,0,0,0,0,0,0
0.994987,0,0,0,0,0,0,0,0,0,0


In [ ]:
(positive_marginal_value * (vote_results > 1).astype(int)).sum(axis=1).hvplot.line(label="1p1v") * positive_marginal_value.sum(axis=1).hvplot.line(label="Optimal", title="Marginal Value of 1p1v vs optimal.")

:Overlay
   .Curve.A_1p1v  :Curve   [funding]   (0)
   .Curve.Optimal :Curve   [funding]   (0)